# Product Recommendation

Generating frequent itemsets for product recommendations using Associal Rules Mining through Apriori Algorithm

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from apyori import apriori

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
# Loading file into dataframe
df_store = pd.read_excel(io='data/Sample_Superstore_Past.xls',sheet_name='Orders',usecols="B:U")
df_store.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
df_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       9994 non-null   object        
 1   Order Date     9994 non-null   datetime64[ns]
 2   Ship Date      9994 non-null   datetime64[ns]
 3   Ship Mode      9994 non-null   object        
 4   Customer ID    9994 non-null   object        
 5   Customer Name  9994 non-null   object        
 6   Segment        9994 non-null   object        
 7   Country        9994 non-null   object        
 8   City           9994 non-null   object        
 9   State          9994 non-null   object        
 10  Postal Code    9994 non-null   int64         
 11  Region         9994 non-null   object        
 12  Product ID     9994 non-null   object        
 13  Category       9994 non-null   object        
 14  Sub-Category   9994 non-null   object        
 15  Product Name   9994 n

In [5]:
df_store[df_store['Order Date'] > '2017-01-01']['Product ID'].nunique()

1524

In [6]:
# Check if there multiple lines of same product ID in one order
#df_store[df_store['Order Date'] > '2022-01-01'].groupby(['Order ID','Product ID'])['Product ID'].count().sort_values(ascending=False) 
s = df_store.groupby('Order ID').count()['Product ID']
s[s == 1]

Order ID
CA-2014-100006    1
CA-2014-100293    1
CA-2014-100328    1
CA-2014-100391    1
CA-2014-100860    1
                 ..
US-2017-166611    1
US-2017-167570    1
US-2017-168613    1
US-2017-168690    1
US-2017-168802    1
Name: Product ID, Length: 2538, dtype: int64

In [7]:
df_store2 = df_store.copy()
df_store2['Product'] = df_store2['Sub-Category'] + ' ' + df_store2['Product Name'].str.split().str.get(0)

df_store2.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Product
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,Bookcases Bush
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,Chairs Hon
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,Labels Self-Adhesive
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,Tables Bretford
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,Storage Eldon


In [8]:
# Adding up same product in one invoice into 1
df_store1 = df_store2.groupby(['Order ID', 'Product']).agg({
    'Order Date': 'first',
    'Ship Date' : 'first',
    'Ship Mode' : 'first',
    'Customer ID': 'first',
    'Customer Name' : 'first',
    'Segment' : 'first',
    'Country' : 'first',
    'City' : 'first',
    'State' : 'first',
    'Postal Code' : 'first',
    'Region' : 'first',
    'Category' : 'first',
    'Sub-Category' : 'first',
    'Product Name' : 'first',
    'Sales' : 'sum',
    'Quantity': 'sum',
    'Discount' : 'first',
    'Profit' : 'sum'
    })

In [9]:
df_store1.reset_index(inplace=True)

In [10]:
dataset = df_store1.groupby('Order ID')['Product'].apply(list)
dataset = dataset[dataset.map(len) > 1].to_list()

In [11]:
dataset[0]

['Binders Wilson', 'Tables Hon']

In [12]:
# association_rules = apriori(dataset, min_support=0.0012,min_confidence=0.5,min_lift=2)
association_rules = apriori(dataset, min_support=0.0012,min_confidence=0.5,min_lift=2)
association_list = list(association_rules)
len(association_list)

29

In [13]:
cols = ['antecedent','antc_desc','consequent','conse_desc','support','confidence','lift',]
df_association = pd.DataFrame(columns=cols)
cnt = 0
for i in association_list:
    cnt = str(cnt)
    df_association.loc[cnt,'antecedent'] = list(i.ordered_statistics[0].items_base)[0]
    df_association.loc[cnt,'consequent'] = list(i.ordered_statistics[0].items_add)[0]
    df_association.loc[cnt,'support'] = round(i.support,5)
    df_association.loc[cnt,'confidence'] = round(i.ordered_statistics[0].confidence,2)
    df_association.loc[cnt,'lift'] = round(i.ordered_statistics[0].lift,2)
    antc_prod_id = list(i.ordered_statistics[0].items_base)[0]
    conse_prod_id = list(i.ordered_statistics[0].items_add)[0]
    df_association.loc[cnt,'antc_desc'] = df_store2[df_store2['Product']==antc_prod_id]['Product Name'].iloc[0]
    df_association.loc[cnt,'conse_desc'] = df_store2[df_store2['Product']==conse_prod_id]['Product Name'].iloc[0]
    cnt = int(cnt)
    cnt +=1
df_association.sort_values(by=['lift','confidence'],ascending=[False,False])

,antecedent,antc_desc,consequent,conse_desc,support,confidence,lift
0,Appliances 1.7,"1.7 Cubic Foot Compact ""Cube"" Office Refrigera...",Binders Avery,Avery Recycled Flexi-View Covers for Binding S...,0.00163,0.67,9.83
5,Binders Peel,Peel & Stick Add-On Corner Pockets,Furnishings Eldon,Eldon Expressions Wood and Plastic Desk Access...,0.00123,0.5,9.79
17,Art Newell,Newell 322,Chairs Global,"Global Deluxe Stacking Chair, Gray",0.00123,0.75,9.41
15,Art Newell,Newell 322,Labels Avery,Avery 485,0.00123,0.75,8.87
26,Furnishings DAX,"DAX Metal Frame, Desktop, Stepped-Edge",Chairs Global,"Global Deluxe Stacking Chair, Gray",0.00123,0.6,7.53
3,Binders Accohide,Accohide Poly Flexible Ring Binders,Labels Avery,Avery 485,0.00163,0.57,6.76
19,Art Newell,Newell 322,Labels Avery,Avery 485,0.00123,0.5,5.91
24,Envelopes Staple,Staple envelope,Paper Xerox,Xerox 1967,0.00123,1.0,4.37
28,Phones Samsung,Samsung Galaxy Note 3,Paper Xerox,Xerox 1967,0.00163,0.8,3.5
13,Supplies Acco,Acco Side-Punched Conventional Columnar Pads,Paper Xerox,Xerox 1967,0.00123,0.75,3.28
